In [ ]:
import csv
import pandas as pd
import numpy as np

datadir = "D:/Python/2045/data/2045-sets-v9.csv"
df_raw = pd.read_csv(datadir)

print(df_raw)

                     Provinsi 2010 GRDP (Ribu Rp)  2011 GRDP  2012 GRDP  \
0                        Aceh            22450.14    22704.8   23099.13   
1              Sumatera Utara            25412.07   26711.24   28036.88   
2              Sumatera Barat            21584.91   22638.75   23744.01   
3                        Riau            69701.03   71637.89   72396.34   
4                       Jambi            29160.16   30856.66   32417.72   
5            Sumatera Selatan               25932   27157.98   28577.89   
6                    Bengkulu            16463.68   17282.27   18143.51   
7                     Lampung            19722.39   20739.31   21794.83   
8   Kepulauan Bangka Belitung            28906.78   30212.18   31172.42   
9              Kepulauan Riau            65703.34   68024.21      70930   
10                DKI Jakarta           111528.86  117672.92  123962.38   
11                 Jawa Barat            20974.94   21976.53      23036   
12                Jawa Te

In [29]:
Tahun = ['2010 IPM', '2011 IPM', '2012 IPM', '2013 IPM', '2014 IPM', '2015 IPM', '2016 IPM', '2017 IPM', '2018 IPM', '2019 IPM', '2020 IPM',
         '2010 Kemiskinan', '2011 Kemiskinan', '2012 Kemiskinan', '2013 Kemiskinan']
for i in Tahun:
    df_raw[i] = df_raw[i].astype(str)
    df_raw[i] = df_raw[i].str.replace(',', '.')
    df_raw[i] = df_raw[i].str.replace(' ', '')  
    df_raw[i] = df_raw[i].replace(['', 'nan', 'None'], np.nan)  
    df_raw[i] = pd.to_numeric(df_raw[i], errors='coerce')

print(df_raw.head())


         Provinsi 2010 GRDP (Ribu Rp) 2011 GRDP 2012 GRDP  2013 GRDP  \
0            Aceh            22450.14   22704.8  23099.13   23228.59   
1  Sumatera Utara            25412.07  26711.24  28036.88   29339.21   
2  Sumatera Barat            21584.91  22638.75  23744.01   24857.64   
3            Riau            69701.03  71637.89  72396.34   72297.05   
4           Jambi            29160.16  30856.66  32417.72   34012.10   

   2014 GRDP  2015 GRDP  2016 GRDP  2017 GRDP  2018 GRDP  ...  \
0   23129.04   22524.31   22835.29   23362.90   24013.79  ...   
1   30477.07   31637.41   32885.09   34183.58   35570.50  ...   
2   25982.83   27080.76   28164.93   29312.17   30470.80  ...   
3   72390.88   70769.78   70569.36   70740.43   70736.77  ...   
4   35878.09   36753.52   37728.80   38833.87   40025.52  ...   

   2011 Kemiskinan  2012 Kemiskinan  2013 Kemiskinan  2014 Kemiskinan  \
0            19.57            19.46            17.72            16.98   
1            11.33            

In [31]:

df_melt = df_raw.melt(id_vars=['Provinsi'], 
                        var_name='Tahun_Indikator', 
                        value_name='Value')

df_melt[['Tahun', 'Indikator']] = df_melt['Tahun_Indikator'].str.split(' ', n=1, expand=True)
df_melted = df_melt.drop('Tahun_Indikator', axis=1)
df_melted = df_melted[['Provinsi', 'Tahun', 'Indikator', 'Value']]
df_final = df_melted.pivot(index=['Provinsi', 'Tahun'], 
                          columns='Indikator', 
                          values='Value').reset_index()
df_final.columns.name = None

print(df_final.head())


  Provinsi Tahun      GRDP GRDP (Ribu Rp)    IPM Inflasi Kemiskinan  \
0     Aceh  2010       NaN       22450.14   71.7     NaN      20.98   
1     Aceh  2011   22704.8            NaN  72.16     NaN      19.57   
2     Aceh  2012  23099.13            NaN  67.81     NaN      19.46   
3     Aceh  2013  23228.59            NaN   68.3     NaN      17.72   
4     Aceh  2014  23129.04            NaN  68.81     NaN      16.98   

  Pengangguran  
0         8.37  
1            9  
2         9.06  
3        10.12  
4         9.02  


In [ ]:
df_final[['IPM', 'RLS']].corr(method= 'pearson')